## Running Excalibur on a Cluster

For real world applications, one typically needs a cross section computed across a grid of temperatures and pressures. Such computations can be much more efficiently handled on a computing cluster, where each (P,T) pair can be assigned to a distributed 'job'.

This tutorial describes how to run `excalibur` on a cluster, focusing on clusters managed by the common Slurm scheduling system.

<div class="alert alert-info">

  **Note:**

  Every computing cluster is special and has its own unique architecture. We strongly recommend reading the documentation for your local cluster before proceeding and adapting the code below accordingly.

</div>

Running on a cluster involves two separate files:

1. A Python file calling `excalibur` (similar to those you've seen in the previous tutorials).
2. A shell script to distribute each pressure-temperature point to a different core.

#### Python script in cluster mode

Let's first create a python file to calculate cross sections for $\mathrm{Fe}$, $\mathrm{Ti}$, $\mathrm{Mg}$, and $\mathrm{Fe^{+}}$ all at once. When a user places `excalibur` in cluster mode (via ``cluster_run = True``) the code will use a single core for each pressure and temperature pair. 

In the example here, a core will first compute the $\mathrm{Fe}$ cross section at a given (P, T), then continue to compute $\mathrm{Ti}$ at the same (P, T) pair and so on. So each core will calculate 4 cross sections at a single (P, T) point and we will use a Slurm shell script to request enough cores to cover all the (P, T) points we desire cross sections for.

Copy the code below into a .py file... how about `many_atoms_on_my_powerful_cluster.py`

In [ ]:
#***** Example script to batch-run EXCALIBUR on a cluster *****#

from excalibur.core import compute_cross_section

species_neutral = ['Fe', 'Ti', 'Mg']   # Fe, Ti, and Mg (neutral atoms)
species_ions = ['Fe']                  # Fe + (the ionization state is entered below)

database = 'VALD'

input_directory = '/PATH_TO_YOUR_LINE_LISTS/input/'      # Change this to point to your line list input folder

P = [1.0e-6, 1.0e-5, 1.0e-4, 1.0e-3, 1.0e-2, 1.0e-1, 1.0e0, 1.0e1, 1.0e2]    # Pressure (bar)
log_P = [-6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0]                  # log10 pressure
T = [100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0,                 # Temperature (K)
     900.0, 1000.0, 1200.0, 1400.0, 1600.0, 1800.0, 2000.0,
     2500.0, 3000.0, 3500.0]

# Create cross section
for i in range(len(species_neutral)):
     compute_cross_section(database = database, species = species_neutral[i], 
                           pressure = P, temperature = T, S_cut = 1.0e-100,
                           input_dir = input_directory, ionization_state = 1,
                           nu_out_min = 200, nu_out_max = 40000, dnu_out = 0.01,
                           verbose = False, N_cores = 1, cluster_run = True)        # The last argument must be True for a cluster run!

#### Slurm shell script

Now we need to create a shell script to assign a core to each (P, T) pair.

From looking at the Python script above, we have 9 pressures and 18 temperatures, for a total of 162 (P, T) pairs. So we will create a shell script to submit 162 jobs, one for each (P, T) point, with a single core being assigned to each job. Each job will run `excalibur` from the terminal via the following commands:

  ```
  python -u many_atoms_on_my_powerful_cluster.py 0
  python -u many_atoms_on_my_powerful_cluster.py 1
  python -u many_atoms_on_my_powerful_cluster.py 2
  .
  .
  .
  python -u many_atoms_on_my_powerful_cluster.py 161
  ```

Where '0' here denotes the first (P, T) pair (P[0], T[0]) and '161' denotes the final (P, T) pair (P[8], T[17]).

To accomplish this, copy the code below into a shell script (.sh). We'll call it `my_ultimate_shell_script.sh`. 

You should also make a folder called `logs` in the same folder to store the terminal output from each job.

In [ ]:
# Job name for the group
JOB_NAME="atoms"

for i in {0..161}; do                     # Loops over the 162 jobs
    srun \
        --account=YOUR_USER_ACCOUNT \             # Your user account on your cluster (or which account will be charged)
        --partition=standard \                    # Your cluster may have a different name for the default partition
        --nodes=1 \
        --cpus-per-task=1 \                       # We only need one core per job, since
        --tasks-per-node=1 \
        --mem=100G \                              # Reserving 100 GB of RAM (less is probably fine)
        --time=0-01:00:00 \                       # Max runtime of 1 hour (atoms will take seconds, large molecules could need days)
        --output=./logs/${JOB_NAME}.$i.out \      # In a directory called 'logs' (make one!), write the terminal output
        --error=./logs/${JOB_NAME}.$i.err \       # Write any error messages into a seperate file
        --job-name=$JOB_NAME \
        python -u /PATH/TO/YOUR/CODE/many_atoms_on_my_powerful_clutster.py $i &     # Path to the Python file above
done

You run this shell script simply by:

  ```
  ./my_ultimate_shell_script.sh
  ```

Congratulations, you have just unlocked the power of calculating cross sections in parallel on 162 cores! 🎉